In [8]:
import os, sys, json
from tqdm import tqdm

In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
%matplotlib inline
# from mpl_toolkits.mplot3d import Axes3D

In [11]:
# bench_dict = {}

def load_json(filename):
    fil = f'./models_v1/stats/{filename}.json'
    if not os.path.exists(fil):
        print(f"File Not Found: {fil}")
        return
    with open(fil) as f:
        data = json.load(f)
    return data

#     bench_dict[model_name] = data
#     return

In [13]:
PREFIX = "03.0_mlp_mixer_"
PREFIX2 = "03.1_mlp_mixer_"

SUFFIX = "_data"

methods = ["dense", "sparseLinear", "sparseMlp"]
dataset = 'c10'
# seeds = [147, 258, 369]
seeds = [147, 258, 369, 741, 852, 963, 159, 357]

hidden_expansion = 2

bench_dict = {}

for method in methods:
    per_method_exps = []
    for seed in seeds:
        file_name_ = f"{method}_l7_{dataset}_s{seed}_h{hidden_expansion}"
        file_name = PREFIX+file_name_+SUFFIX
        
        if method == "sparseLinear":
            file_name = PREFIX2+file_name_+SUFFIX
        
        data = load_json(file_name)
        
        # epoch, loss, accuracy, time
        train_stat = np.array(data["train_stat"])
        test_stat = np.array(data["test_stat"])
        
        stat = {'train_acc':np.max(train_stat[:, 2]), 
                'test_acc':np.max(test_stat[:, 2]), 
                'params':data["num_params"]}
        per_method_exps.append(stat)
        
    best_seed = np.argmax([exp["test_acc"] for exp in per_method_exps])
    bench_dict[method] = per_method_exps[best_seed]
    
    accs = [exp["test_acc"] for exp in per_method_exps]
    print(method, np.mean(accs), np.std(accs),  np.max(accs))

dense 84.07000000000001 0.3554926159570698 84.47
sparseLinear 83.48875000000001 0.32200300852631886 83.84
sparseMlp 83.69875 0.30168019076499014 84.26


In [ ]:
# ./models_v1/stats/03.0_mlp_mixer_dense_l7_c10_s147_h1_data.json

In [ ]:
bench_dict

## Final Benchmark

In [ ]:
"""
H1:
dense 83.83250000000001 0.4065940850528961 84.64
sparseLinear 82.45375000000001 0.36065695265723324 83.01
sparseMlp 82.68 0.2444381312316071 83.06


H2:
dense 84.07000000000001 0.3554926159570698 84.47
sparseLinear 83.48875000000001 0.32200300852631886 83.84
sparseMlp 83.69875 0.30168019076499014 84.26

"""

## New

In [ ]:
"""
### H2 -> New Method
dense 83.89 0.37585458180879894 84.19
sparseLinear 83.45666666666666 0.4598792111945142 83.84
sparseMlp 83.75333333333333 0.16759740119968922 83.98

### H2 -> OLD Method (debugged)
sparseLinear 83.45666666666666 0.07318166133366749 83.56

### H1
dense 83.76333333333334 0.3318969050105132 84.06
sparseLinear 82.59666666666665 0.41587925598129727 83.14
sparseMlp 82.53333333333335 0.2422578974747561 82.87

"""

### Old - Linear Bug

In [7]:
## All seeds
### H1
"""
dense 83.83250000000001 0.4065940850528961 84.64
sparseLinear 82.40375 0.29406366232501185 82.82
sparseMlp 82.68 0.2444381312316071 83.06
"""
### H2
"""
dense 84.07000000000001 0.3554926159570698 84.47
sparseLinear 82.40125 0.3260919157231609 82.93
sparseMlp 83.69875 0.30168019076499014 84.26
"""


""

''